# Web Scraping en Python: Ejemplo práctico (I)

# Web Scraping en Python: Ejemplo práctico

extraer la información de la portada de la edición digital del periódico "El País".

### Parte 0: El objetivo

tendrás que saber qué buscas.
el objetivo
extraer los titulares de las noticias del día, la sección a la que pertenecen y la fecha de las mismas y luego volcarlo todo en un `DataFrame`

### Parte I: Primera inspección: sobre el terreno

Pues ya lo tenemos necesitamos actuar sobre los Tags:
* time, para obtener la fecha, cuyo abuelo es un div, con el data dtm region, portada menú para obtener la fecha
* section, para obtener la sección, buscar todos los section, con su atributo data dtm region, para obtner las secciones
* article, para obtener el titular, dentro de cada sección buyscar los tags articles para tener el tiular.

sabemos que tenemos que aplicar algunos filtros para no equivocarnos de dato, o por lo menos asegurarnos de que no lo hacemos (por ejemplo con time) y tenemos que decidir si no hay información. Es decir toca inspeccionar sobre plano.

### Parte II: Descarga de la página

In [2]:

import pandas as pd
import requests

from bs4 import BeautifulSoup

#### Inciso: carga de páginas web, métodos http, códigos de respuesta

para cargar una página web usamos la función `request` del módulo requests:

Descargar:

In [4]:
# Version 1, larga:
url = "https://static.elpais.com/hemeroteca/elpais/2023/04/24/m/portada.html"
elpais = requests.request(url = url, method = "GET") #descargar la dirección, nos devuelve un objetop especial donde está la página web

#### Métodos

<table class="table table-hover">
<thead>
<tr>
<th>HTTP method</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>GET</code></td>
<td>Obtiene un recurso existente (ahora lee página web en vez de recurso).</td>
</tr>
<tr>
<td><code>POST</code></td>
<td>Crea un nuevo recurso (ahora lee página web en vez de recurso)</td>
</tr>
<tr>
<td><code>PUT</code></td>
<td>Actualiza un recurso existente (pues eso lee página web ...).</td>
</tr>
<tr>
<td><code>DELETE</code></td>
<td>Borra un recurso (pues idem.)</td>
</tr>
</tbody>
</table>

In [5]:
print(type(elpais)) #nos devuelve un objeto tipo Response

<class 'requests.models.Response'>


objeto tipo Response,
conviene saber
siguientes atributos y métodos (los usaremos cuando hablemos de APIs):
1. **`.status_code`**: (código de vuelta como un 404 not found)
   - Representa el código de estado HTTP de la respuesta. Por ejemplo, 200 para éxito, 404 para no encontrado, 500 para error interno del servidor, etc.
   - Es útil para verificar si una solicitud se realizó con éxito. Sí, este es el atributo que refleja el famoso 404. Los códigos van por grupos (los 2xx son éxito, 4xx es un problema del cliente, 5xx es un problema del servidor)
2. **`.text`**(texto, código html):
   - Contiene el contenido de la respuesta en formato Unicode. En nuestro caso, es el código HTML.
   - Se utiliza para obtener el cuerpo de la respuesta como una cadena de texto, que es útil para respuestas de tipo texto como HTML, XML, JSON, etc.
3. **`.json()`** (método), nos devuelve un diccionario con la respuesta de verdad de las APIs:
   - Método que convierte la respuesta, asumiendo que es de tipo JSON, a un objeto Python (usualmente un diccionario).
   - Es muy útil cuando se trabaja con APIs que devuelven respuestas en formato JSON. (por eso te lo presento aquí pero lo veremos en las siguientes sesiones)


forma corta de obtener una pagina HTML:

In [6]:
url = "https://static.elpais.com/hemeroteca/elpais/2023/04/24/m/portada.html"
elpais = requests.get(url) #para cuando no solo queremos una página htl, si no que queremos entrar también en una API
#texto html, cuando njos lo da en text

In [7]:
elpais.text

'<!DOCTYPE html><html lang="es-ES"><head><link rel="preconnect" href="//static.elpais.com"><link rel="preconnect" href="//ep00.epimg.net"><link rel="preconnect" href="//imagenes.elpais.com"><link rel="preconnect" href="//www.googletagservices.com"><link rel="preconnect" href="//ping.chartbeat.net"><link rel="preconnect" href="//cdn.krxd.net"><link rel="preconnect" href="//assets.adobedtm.com"><link rel="preload" href="https://static.elpais.com/dist/resources/fonts/majrit/majrit-text/Majrit-Text-Bold.woff2" as="font" type="font/woff2" crossorigin><link rel="preload" href="https://static.elpais.com/dist/resources/fonts/majrit/majrit-text/Majrit-Text-Roman.woff2" as="font" type="font/woff2" crossorigin><link rel="preload" href="https://static.elpais.com/dist/resources/fonts/marcin-ant-b/marcinantb-regular-webfont.woff2" as="font" type="font/woff2" crossorigin><link rel="preload" href="https://static.elpais.com/dist/resources/fonts/majrit/majrit-text/Majrit-Text-Bold-Italic.woff2" as="font

Ahora es cuando "parseamos" la página con BeautifulSoup:

In [8]:
soup_elpais = BeautifulSoup(elpais.text, "lxml") #con esto podemos usar lo aprendido en loas otras lecciones para poder trabajar sobre el texto,
#para obtener lo que queremos.

# Web Scraping en Python: Ejemplo práctico (II)

In [7]:
import pandas as pd
import requests

from bs4 import BeautifulSoup

url = "https://static.elpais.com/hemeroteca/elpais/2023/04/24/m/portada.html"
elpais = requests.get(url) # Obtenemos la página dentro de un objeto Request

soup_elpais = BeautifulSoup(elpais.text, "lxml") # Parseamos el documento HTML de la página

+ **Objetivo**: Obtener las noticias por sección y con su fecha.
+ **Pesquisas**: Tenemos la infor en las etiquetas: time, section y article. Tenemos que hacer un pequeño tratamiento.

podemos pintarnos toda la página con el método `prettify` e intentar hacer el mismo análisis

In [2]:
print(soup_elpais.prettify())

<!DOCTYPE html>
<html lang="es-ES">
 <head>
  <link href="//static.elpais.com" rel="preconnect"/>
  <link href="//ep00.epimg.net" rel="preconnect"/>
  <link href="//imagenes.elpais.com" rel="preconnect"/>
  <link href="//www.googletagservices.com" rel="preconnect"/>
  <link href="//ping.chartbeat.net" rel="preconnect"/>
  <link href="//cdn.krxd.net" rel="preconnect"/>
  <link href="//assets.adobedtm.com" rel="preconnect"/>
  <link as="font" crossorigin="" href="https://static.elpais.com/dist/resources/fonts/majrit/majrit-text/Majrit-Text-Bold.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://static.elpais.com/dist/resources/fonts/majrit/majrit-text/Majrit-Text-Roman.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://static.elpais.com/dist/resources/fonts/marcin-ant-b/marcinantb-regular-webfont.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://static.elpais.com/dist/resources

Nos es más fácil buscar así ,métodos específicos, sobretodo usanmdp el findall y esos métodos.

### Extración de la fecha: Tag -> time

Sabemos que la fecha está en un Tag "time", pero vimos que había varios, tenemos que asegurarnos que es el Tag "time" que cuelga de otro Tag con atributo "data-dtm-region" igual a "portada_menu". Entonces tenemos que escribirnos un código que nos capture precisamente ese Tag time:

In [11]:
# Iremos recuperando todos los Tags de tipo "time" y para cada uno recorreremos el árbol de sus padres (parents) con el método
# correspondiente, cuando encontremos el que tenga el atributo indicado, nos quedaremos con el Tag "time" que 
# estábamos revisando y pararemos
fin = False #es un flag que activaremos para poder salirnos del anidado de for
for tiempo in soup_elpais.find_all("time"): #Buscamos todas las etiquetas time y después padres, abuelos, etc.
    for padre in tiempo.parents:
        if padre.attrs.get("data-dtm-region", None) == "portada_menu":   # entre sus atributos está data dtm region, que si no está nos da none. Usamos el get de los diccionarios. Busca el atributo en todos los tags parents
            print(tiempo)
            tiempo_tag = tiempo #creamos variable tag, para conservar el tiempo yh usarlo cuando construyamos el df
            fin = True
            break #los breaks rompen el bucle más cercano
    if fin:
        break #me dice que vengo de ese break y hago otro para salirme delo otro.

<time data-date="2023-04-24T09:51:53.591Z" datetime="2023-04-24T09:51:53.591Z" id="header_date_88"><span>24 abr 2023</span><span class="x_e_s">|</span><span>Actualizado<!-- --> <!-- -->09:51<!-- --> <abbr title="UTC">UTC</abbr></span><span class="x_e_s">|</span></time>


Nos da el valor en los atributos

In [12]:
print(tiempo.text)

24 abr 2023|Actualizado 09:51 UTC|


¿Y que valor me quedo? Veamos los atributos, string y text:

In [13]:
print("Atributos:",tiempo.attrs)
print("String:", tiempo.string)
print("text:", tiempo.text)

Atributos: {'id': 'header_date_88', 'datetime': '2023-04-24T09:51:53.591Z', 'data-date': '2023-04-24T09:51:53.591Z'}
String: None
text: 24 abr 2023|Actualizado 09:51 UTC|


In [ ]:
Nos quedamos con el atributo datetime, ya que en pandas lo convertimos y lo use como tal

In [ ]:
Podemos escoger el que queramos, vamos a quedarnos el datetime porque luego le diremos a Pandas que lo use como tal:

In [14]:
fecha = tiempo_tag["datetime"] #atributo datetime, y se accede como un diccionario

### Extracción de los artículos y las secciones: Tags -> section y article

In [ ]:
Tenemos el tiempo es el momento de capturar los artículos y sus secciones.

In [15]:
# Recorramos todos los Tags tipo "section", capturemos su atributo "data-dtm-regioin" y luego para cada "section" vayamos acumulando sus artículos
for seccion in soup_elpais.find_all("section"):
    nombre_seccion = seccion.get("data-dtm-region", "Otros")  #recuperamos de seccion con el get y si no tiene atributos o no tienen una sección definida, se agrupan en otros
    for articulo in seccion.find_all("article"):
        noticia = articulo.header.get_text(separator = ":")  #noticia es igual al header y nos quedamos con todos los textos y los separamos con el :
        if noticia: 
            print(f"He encontrado una noticia de la sección {nombre_seccion}:")
            print(f"NOT -> {noticia}")
            print("\n\n")


He encontrado una noticia de la sección portada_apertura:
NOT -> vídeo en directo:La exhumación de Primo de Rivera se lleva a cabo a puerta cerrada 



He encontrado una noticia de la sección portada_apertura:
NOT -> El Gobierno ejecuta el traslado de los restos del fundador de la Falange tras los de Franco, Queipo, Sanjurjo y Mola



He encontrado una noticia de la sección portada_apertura:
NOT -> Vídeo | Así ‘sacralizó’ el franquismo al líder falangista:Samuel Martínez



He encontrado una noticia de la sección portada_apertura:
NOT -> El gasto militar en Europa alcanza niveles de la Guerra Fría



He encontrado una noticia de la sección portada_apertura:
NOT -> El general ucranio Serguéi Melnik: “Solo los rusos pueden cambiar Rusia. Será entonces cuando termine la guerra”



He encontrado una noticia de la sección portada_apertura:
NOT -> EDITORIAL:Pulso agrícola en la UE 



He encontrado una noticia de la sección portada_apertura:
NOT -> Una moratoria artificial



He encontrado u

### Juntándolo todo en un `DataFrame`

In [16]:
# Creamos nuestra lista de diccionarios, uno por noticia
lista_noticias = []
for seccion in soup_elpais.find_all("section"):
    nombre_seccion = seccion.get("data-dtm-region", "Otros")  
    for articulo in seccion.find_all("article"):
        noticia = articulo.header.get_text(separator = ":")  
        if noticia: 
           lista_noticias.append({"fecha":fecha, "noticia":noticia, "seccion": nombre_seccion}) #añade lista noticias como un diccionario y podemos usar pandas

In [17]:
df = pd.DataFrame(lista_noticias)
df

,fecha,noticia,seccion
0,2023-04-24T09:51:53.591Z,vídeo en directo:La exhumación de Primo de Riv...,portada_apertura
1,2023-04-24T09:51:53.591Z,El Gobierno ejecuta el traslado de los restos ...,portada_apertura
2,2023-04-24T09:51:53.591Z,Vídeo | Así ‘sacralizó’ el franquismo al líder...,portada_apertura
3,2023-04-24T09:51:53.591Z,El gasto militar en Europa alcanza niveles de ...,portada_apertura
4,2023-04-24T09:51:53.591Z,El general ucranio Serguéi Melnik: “Solo los r...,portada_apertura
...,...,...,...
130,2023-04-24T09:51:53.591Z,Un médico triunfa al contar el comentario que ...,Otros
131,2023-04-24T09:51:53.591Z,Asensio depende de Brahim,Otros
132,2023-04-24T09:51:53.591Z,Del café a la cocaína en el trabajo: la otra c...,Otros
133,2023-04-24T09:51:53.591Z,Las 10 mejores películas y series de mafia de ...,Otros


Para terminar convertimos el campo "fecha" en `datetime` [aunque aquí parece poco útil, lo será si repetimos esta operación todos los días o con toda la hemeroteca], y guardamos nuestro `DataFrame` [Nota: si sientes curiosidad el formato de fecha que usa "El País" es el denominado ISO-8601]

In [19]:
df["fecha"] = pd.to_datetime(df["fecha"])
df

,fecha,noticia,seccion
0,2023-04-24 09:51:53.591000+00:00,vídeo en directo:La exhumación de Primo de Riv...,portada_apertura
1,2023-04-24 09:51:53.591000+00:00,El Gobierno ejecuta el traslado de los restos ...,portada_apertura
2,2023-04-24 09:51:53.591000+00:00,Vídeo | Así ‘sacralizó’ el franquismo al líder...,portada_apertura
3,2023-04-24 09:51:53.591000+00:00,El gasto militar en Europa alcanza niveles de ...,portada_apertura
4,2023-04-24 09:51:53.591000+00:00,El general ucranio Serguéi Melnik: “Solo los r...,portada_apertura
...,...,...,...
130,2023-04-24 09:51:53.591000+00:00,Un médico triunfa al contar el comentario que ...,Otros
131,2023-04-24 09:51:53.591000+00:00,Asensio depende de Brahim,Otros
132,2023-04-24 09:51:53.591000+00:00,Del café a la cocaína en el trabajo: la otra c...,Otros
133,2023-04-24 09:51:53.591000+00:00,Las 10 mejores películas y series de mafia de ...,Otros
